import packages

In [ ]:
import numpy as np 
import os 
import sys
from pathlib import Path 
import pickle
from ase.io import read
from ase.data import vdw_radii
from ase.units import _amu

path_to_prn = os.path.abspath(
    '.../PoroNet')
sys.path.append(path_to_prn)
import poronet_functions as prn

Import the required MOFs and pore index

In [ ]:
#Here we chose the pores with diameter from 8.4 to 8.7 as examples
Required_MOFs=pickle.load(open(('Required_MOFs_84_87.pkl'), 'rb'))
Index_of_pbc_pores_84_87=pickle.load(open(('Index_of_pbc_pores_84_87.pkl'), 'rb'))

Calculte volumetric surface area with loop structure

In [ ]:
pbc_area_m2_L_list=[]
error=[]
for i,MOF in enumerate(Required_MOFs):
  try:
    #set the path of the cif files
    cif_path = f".../Tobacco/{MOF}.cif"
    mat_atoms = read(cif_path)
    dgrid = prn.dgrid_from_atoms_cpu_no_aabb(mat_atoms, spacing=0.5)
    regions, maxima = prn.regions_from_dgrid_with_threshold_abs(dgrid, mask_thickness = 0, h=0.5, threshold_abs=1)
    maxima_radii = dgrid[tuple(maxima.T )]
    pbc_groups = prn.apply_pbc(regions, maxima, maxima_radii, mat_atoms, return_conn=True)
    regions_pbc = prn.apply_pbc(regions, maxima, maxima_radii, mat_atoms, return_conn=False)
    probe_size = vdw_radii[2]
    mass_g = mat_atoms.get_masses().sum() * _amu * 1000  # this is in grams
    dist_iso_submeshes =prn.get_isosurface_submeshes(dgrid,probe_size,mat_atoms, regions_pbc, colorscale=None)
    group_areas = [(m.area * 1e-20) for m in dist_iso_submeshes]#m2
    j=Index_of_pbc_pores_84_87[i]
    count_j = np.sum(regions_pbc.flatten() == j+1)
    count_all = len(regions_pbc.flatten())
    ratio_j=count_j/count_all
    pbc_area_m2=group_areas[j]
    cell_volume=mat_atoms.get_volume()
    cell_volume_L=cell_volume/(1E27)
    pore_volume_L=cell_volume_L*ratio_j
    pbc_area_m2_L=pbc_area_m2/pore_volume_L
    pbc_area_m2_L_list.append(pbc_area_m2_L)
    print(f"{i+1}/{len(Required_MOFs)}")
  except Exception as e:
        error.append(i)
        continue